In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import time
from zipfile import ZipFile
import warnings
import pickle
import torch
from torch.utils.data import Dataset
import tsfel
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm
import lightgbm as lgb
from hiclass import LocalClassifierPerNode, LocalClassifierPerParentNode, LocalClassifierPerLevel
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
warnings.filterwarnings('ignore')

In [21]:
base_dir = "../logs/ensemble/base_ensemble/01_27_2025-12_43_22"

In [22]:
os.listdir(base_dir)

['xgb', 'rf', 'lgb']

In [ ]:
"""
TODO:
1. 
"""

In [23]:
cv_res = pd.read_csv(os.path.join(base_dir, 'rf/cv_report.csv'))
cv_res

,col,precision,recall,f1,support
0,Active_Power_Sensor,0.174897,0.273752,0.213434,NaN
1,Air_Flow_Sensor,0.182297,0.272370,0.218411,NaN
2,Air_Flow_Setpoint,0.098485,0.144444,0.117117,NaN
3,Air_Temperature_Sensor,0.264496,0.298428,0.280440,NaN
4,Air_Temperature_Setpoint,0.232540,0.277473,0.253027,NaN
...,...,...,...,...,...
90,Zone_Air_Dewpoint_Sensor,0.301282,0.261111,0.279762,NaN
91,Zone_Air_Humidity_Sensor,0.318627,0.300926,0.309524,NaN
92,Zone_Air_Humidity_Setpoint,0.000000,0.000000,0.000000,NaN
93,Zone_Air_Temperature_Sensor,0.289678,0.294362,0.292001,NaN


In [27]:
avg = [
    pd.read_csv(os.path.join(base_dir, "rf/test_predictions", f"final_result_{i}.csv"))
    for i in range(2)
]


In [29]:
import pandas as pd

def merge_mean(df_list):
    """
    Given a list of DataFrames with identical columns and index,
    return one DataFrame whose numeric columns are the mean of all
    those numeric columns across the list.

    The "filename" column is assumed to be the same or can be taken
    from the first DataFrame.
    """
    # Make a copy of the first DataFrame to initialize the output
    merged_df = df_list[0].copy()

    # Identify all numeric columns (excluding "filename")
    numeric_cols = merged_df.columns.drop("filename")

    # Sum up the numeric columns from all other DataFrames
    for df in df_list[1:]:
        merged_df[numeric_cols] += df[numeric_cols]

    # Divide by the number of DataFrames to get the mean
    merged_df[numeric_cols] /= len(df_list)

    return merged_df


# Example usage:
# Suppose dfs is the list of DataFrames you showed in the question
# final_df = merge_mean(dfs)
# print(final_df)


In [36]:
def check_pred_num(_final_res, thr=0.4):
    # Exclude 'filename' column if it exists
    filtered_df = _final_res.drop(columns=['filename'], errors='ignore')

    return (filtered_df >= thr).sum(axis=1)

In [37]:
m_res = merge_mean(avg)

In [38]:
check_pred_num(m_res).value_counts()

2     85307
3     74616
4     58612
1     52155
5     31619
6      9940
7      2610
8       726
9       119
10       16
Name: count, dtype: int64

In [39]:
from zipfile import ZipFile
zipftest = ZipFile('../downloads/test_X_v0.1.0.zip', 'r')
listtestfile = zipftest.namelist()[1:]

In [41]:
columnlist = [
    'Active_Power_Sensor', 'Air_Flow_Sensor',
    'Air_Flow_Setpoint', 'Air_Temperature_Sensor',
    'Air_Temperature_Setpoint', 'Alarm', 'Angle_Sensor',
    'Average_Zone_Air_Temperature_Sensor',
    'Chilled_Water_Differential_Temperature_Sensor',
    'Chilled_Water_Return_Temperature_Sensor',
    'Chilled_Water_Supply_Flow_Sensor',
    'Chilled_Water_Supply_Temperature_Sensor', 'Command',
    'Cooling_Demand_Sensor', 'Cooling_Demand_Setpoint',
    'Cooling_Supply_Air_Temperature_Deadband_Setpoint',
    'Cooling_Temperature_Setpoint', 'Current_Sensor',
    'Damper_Position_Sensor', 'Damper_Position_Setpoint', 'Demand_Sensor',
    'Dew_Point_Setpoint', 'Differential_Pressure_Sensor',
    'Differential_Pressure_Setpoint',
    'Differential_Supply_Return_Water_Temperature_Sensor',
    'Discharge_Air_Dewpoint_Sensor', 'Discharge_Air_Temperature_Sensor',
    'Discharge_Air_Temperature_Setpoint',
    'Discharge_Water_Temperature_Sensor', 'Duration_Sensor',
    'Electrical_Power_Sensor', 'Energy_Usage_Sensor',
    'Filter_Differential_Pressure_Sensor', 'Flow_Sensor', 'Flow_Setpoint',
    'Frequency_Sensor', 'Heating_Demand_Sensor', 'Heating_Demand_Setpoint',
    'Heating_Supply_Air_Temperature_Deadband_Setpoint',
    'Heating_Temperature_Setpoint', 'Hot_Water_Flow_Sensor',
    'Hot_Water_Return_Temperature_Sensor',
    'Hot_Water_Supply_Temperature_Sensor', 'Humidity_Setpoint',
    'Load_Current_Sensor', 'Low_Outside_Air_Temperature_Enable_Setpoint',
    'Max_Air_Temperature_Setpoint', 'Min_Air_Temperature_Setpoint',
    'Outside_Air_CO2_Sensor', 'Outside_Air_Enthalpy_Sensor',
    'Outside_Air_Humidity_Sensor',
    'Outside_Air_Lockout_Temperature_Setpoint',
    'Outside_Air_Temperature_Sensor', 'Outside_Air_Temperature_Setpoint',
    'Parameter', 'Peak_Power_Demand_Sensor', 'Position_Sensor',
    'Power_Sensor', 'Pressure_Sensor', 'Rain_Sensor',
    'Reactive_Power_Sensor', 'Reset_Setpoint',
    'Return_Air_Temperature_Sensor', 'Return_Water_Temperature_Sensor',
    'Room_Air_Temperature_Setpoint', 'Sensor', 'Setpoint',
    'Solar_Radiance_Sensor', 'Speed_Setpoint', 'Static_Pressure_Sensor',
    'Static_Pressure_Setpoint', 'Status', 'Supply_Air_Humidity_Sensor',
    'Supply_Air_Static_Pressure_Sensor',
    'Supply_Air_Static_Pressure_Setpoint', 'Supply_Air_Temperature_Sensor',
    'Supply_Air_Temperature_Setpoint', 'Temperature_Sensor',
    'Temperature_Setpoint', 'Thermal_Power_Sensor', 'Time_Setpoint',
    'Usage_Sensor', 'Valve_Position_Sensor', 'Voltage_Sensor',
    'Warmest_Zone_Air_Temperature_Sensor', 'Water_Flow_Sensor',
    'Water_Temperature_Sensor', 'Water_Temperature_Setpoint',
    'Wind_Direction_Sensor', 'Wind_Speed_Sensor',
    'Zone_Air_Dewpoint_Sensor', 'Zone_Air_Humidity_Sensor',
    'Zone_Air_Humidity_Setpoint', 'Zone_Air_Temperature_Sensor'
]

In [48]:
import numpy as np

pred_idx = np.load("../logs/0127_xgb_base.npy.0")
pred_res = np.zeros((len(listtestfile), len(columnlist)))
pred_res[pred_idx[0], pred_idx[1]] = 1.0

In [49]:
semi_pred_filename = pd.DataFrame(columns=['filename'])
semi_pred_filename['filename'] = pd.Series(listtestfile).apply(lambda x: x.split("/")[-1])

semi_pred_res = pd.DataFrame(data=pred_res, columns=columnlist)

semi_pred_res = pd.concat([semi_pred_filename, semi_pred_res], axis=1)

In [50]:
check_pred_num(semi_pred_res, thr=0.4).value_counts()

1     100659
2      82777
3      76259
4      37805
5      13446
0       3137
6       1256
7        275
8         81
9         23
10         2
Name: count, dtype: int64

# Occurence post filtering

In [141]:
LEVEL_LABLES = [
    ['Alarm', 'Command', 'Parameter', 'Sensor', 'Setpoint', 'Status'],
    ['Reset_Setpoint',
    'Usage_Sensor',
    'Pressure_Sensor',
    'Flow_Setpoint',
    'Static_Pressure_Setpoint',
    'Angle_Sensor',
    'Humidity_Setpoint',
    'Temperature_Sensor',
    'Temperature_Setpoint',
    'Supply_Air_Humidity_Sensor',
    'Outside_Air_CO2_Sensor',
    'Differential_Pressure_Setpoint',
    'Damper_Position_Setpoint',
    'Heating_Demand_Setpoint',
    'Cooling_Demand_Setpoint',
    'Current_Sensor',
    'Wind_Speed_Sensor',
    'Flow_Sensor',
    'Dew_Point_Setpoint',
    'Zone_Air_Dewpoint_Sensor',
    'Power_Sensor',
    'Position_Sensor',
    'Solar_Radiance_Sensor',
    'Duration_Sensor',
    'Time_Setpoint',
    'Discharge_Air_Dewpoint_Sensor',
    'Wind_Direction_Sensor',
    'Voltage_Sensor',
    'Zone_Air_Humidity_Sensor',
    'Demand_Sensor',
    'Speed_Setpoint',
    'Rain_Sensor',
    'Frequency_Sensor',
    'Outside_Air_Humidity_Sensor',
    'Outside_Air_Enthalpy_Sensor'],
    ['Air_Flow_Sensor',
    'Water_Temperature_Setpoint',
    'Water_Flow_Sensor',
    'Electrical_Power_Sensor',
    'Zone_Air_Humidity_Setpoint',
    'Heating_Temperature_Setpoint',
    'Air_Flow_Setpoint',
    'Energy_Usage_Sensor',
    'Supply_Air_Static_Pressure_Setpoint',
    'Air_Temperature_Sensor',
    'Valve_Position_Sensor',
    'Cooling_Temperature_Setpoint',
    'Water_Temperature_Sensor',
    'Load_Current_Sensor',
    'Damper_Position_Sensor',
    'Static_Pressure_Sensor',
    'Air_Temperature_Setpoint',
    'Thermal_Power_Sensor',
    'Differential_Pressure_Sensor'],
    ['Supply_Air_Temperature_Sensor',
    'Discharge_Air_Temperature_Sensor',
    'Discharge_Water_Temperature_Sensor',
    'Zone_Air_Temperature_Sensor',
    'Supply_Air_Static_Pressure_Sensor',
    'Outside_Air_Temperature_Setpoint',
    'Supply_Air_Temperature_Setpoint',
    'Chilled_Water_Supply_Flow_Sensor',
    'Chilled_Water_Supply_Temperature_Sensor',
    'Peak_Power_Demand_Sensor',
    'Room_Air_Temperature_Setpoint',
    'Hot_Water_Supply_Temperature_Sensor',
    'Active_Power_Sensor',
    'Min_Air_Temperature_Setpoint',
    'Return_Air_Temperature_Sensor',
    'Hot_Water_Flow_Sensor',
    'Chilled_Water_Differential_Temperature_Sensor',
    'Filter_Differential_Pressure_Sensor',
    'Max_Air_Temperature_Setpoint',
    'Cooling_Supply_Air_Temperature_Deadband_Setpoint',
    'Outside_Air_Temperature_Sensor',
    'Heating_Supply_Air_Temperature_Deadband_Setpoint',
    'Discharge_Air_Temperature_Setpoint',
    'Return_Water_Temperature_Sensor',
    'Reactive_Power_Sensor'],
    ['Low_Outside_Air_Temperature_Enable_Setpoint',
    'Cooling_Demand_Sensor',
    'Chilled_Water_Return_Temperature_Sensor',
    'Average_Zone_Air_Temperature_Sensor',
    'Warmest_Zone_Air_Temperature_Sensor',
    'Heating_Demand_Sensor',
    'Differential_Supply_Return_Water_Temperature_Sensor',
    'Hot_Water_Return_Temperature_Sensor',
    'Outside_Air_Lockout_Temperature_Setpoint']
]

In [142]:
tiers = {i+1: LEVEL_LABLES[i] for i in range(len(LEVEL_LABLES))}

def get_tier(label):
    for tier_num, tier_list in tiers.items():
        if label in tier_list:
            return tier_num
    return None  # Handle cases where the label isn't found in any tier

In [143]:
train_y = pd.read_csv("../downloads/train_y_v0.1.0.csv")

In [144]:
stackedfinalresult = semi_pred_res.copy()

In [145]:
check = stackedfinalresult
occurence = []

for col1 in tqdm(list(check.columns)[1:]):
    for col2 in list(check.columns)[1:]:
        if col1 != col2:
            oc = check[[col1, col2]]
            oc = oc[(oc[col1] == 1) & (oc[col2] == 1)]
            occurence.append([col1, col2, len(oc)])

100%|██████████| 94/94 [00:20<00:00,  4.62it/s]


In [146]:
trn_oc_check = train_y.copy()
trn_occurence = []

for col1 in tqdm(list(trn_oc_check.columns)[1:]):
    for col2 in list(trn_oc_check.columns)[1:]:
        if col1 != col2:
            oc = trn_oc_check[[col1, col2]]
            oc = oc[(oc[col1] == 1) & (oc[col2] == 1)]
            trn_occurence.append([col1, col2, len(oc)])

  0%|          | 0/94 [00:00<?, ?it/s]

100%|██████████| 94/94 [00:11<00:00,  8.11it/s]


In [147]:
trn_oc = pd.DataFrame(trn_occurence)
trn_oc = trn_oc[trn_oc[2] > 0]

In [148]:
trn_oc.values[:, :2].shape

oc_map = {}
for src, tgt in trn_oc.values[:, :2]:
    if src not in oc_map:
        oc_map[src] = [tgt]
    else:
        oc_map[src].append(tgt)

In [149]:
tst_oc = pd.DataFrame(occurence)
tst_oc = tst_oc[tst_oc[2] > 0]

In [150]:
rm_label = []
for row in tqdm(tst_oc.values, total=len(tst_oc)):
    if oc_map.get(row[0]) and row[1] not in oc_map.get(row[0]):
        if get_tier(row[0]) < get_tier(row[1]):
            rm_label.append((row[0], row[1]))
        else:
            rm_label.append((row[1], row[0]))

100%|██████████| 2266/2266 [00:00<00:00, 285105.98it/s]


In [167]:
filtered_res = semi_pred_res.copy()
remove_record = []
for src, tgt in tqdm(rm_label):
    remove_size = len(filtered_res[(filtered_res[src] == 1) & (filtered_res[tgt] == 1)])

    # Skip if it removes too much
    skipped = False
    if remove_size > len(filtered_res[filtered_res[tgt] == 1]) * 0.01:
        skipped = True

    # Count how many rows match the condition
    remove_record.append((src, tgt, remove_size, len(filtered_res[filtered_res[tgt] == 1]), skipped))
    
    if not skipped:
        # Properly update the matching rows in the original DataFrame
        filtered_res.loc[(filtered_res[src] == 1) & (filtered_res[tgt] == 1), tgt] = 0.0

100%|██████████| 1798/1798 [00:36<00:00, 49.85it/s] 


In [168]:
rm_check = pd.DataFrame(remove_record)

In [169]:
rm_check

,0,1,2,3,4
0,Air_Flow_Sensor,Active_Power_Sensor,3,7446,False
1,Air_Temperature_Sensor,Active_Power_Sensor,2,7443,False
2,Angle_Sensor,Active_Power_Sensor,15,7441,False
3,Current_Sensor,Active_Power_Sensor,43,7426,False
4,Demand_Sensor,Active_Power_Sensor,14,7383,False
...,...,...,...,...,...
1793,Supply_Air_Temperature_Setpoint,Zone_Air_Temperature_Sensor,20,10640,False
1794,Temperature_Setpoint,Zone_Air_Temperature_Sensor,0,10620,False
1795,Usage_Sensor,Zone_Air_Temperature_Sensor,0,10620,False
1796,Water_Temperature_Sensor,Zone_Air_Temperature_Sensor,0,10620,False


In [170]:
rm_check[4].value_counts()

4
False    1341
True      457
Name: count, dtype: int64

In [171]:
rm_check[[1, 2]].groupby(1).agg('sum').sort_values(2, ascending=False)

,2
1,
Electrical_Power_Sensor,9003
Position_Sensor,7686
Damper_Position_Sensor,7045
Sensor,6864
Peak_Power_Demand_Sensor,5236
...,...
Discharge_Air_Temperature_Setpoint,2
Hot_Water_Flow_Sensor,2
Solar_Radiance_Sensor,0


In [ ]:
# 1     100659
# 2      82777
# 3      76259
# 4      37805
# 5      13446
# 0       3137
# 6       1256
# 7        275
# 8         81
# 9         23
# 10         2

# 1    103552
# 2     86048
# 3     75239
# 4     35331
# 5     12143
# 0      3137
# 6       270

In [173]:
check_pred_num(filtered_res, thr=0.4).value_counts()

1    103552
2     86048
3     75239
4     35331
5     12143
0      3137
6       270
Name: count, dtype: int64

In [174]:
filtered_res.to_csv("../logs/submit/0127_xgb_base_post_filter_v2.csv", index=False)

In [52]:
semi_pred_res

,filename,Active_Power_Sensor,Air_Flow_Sensor,Air_Flow_Setpoint,Air_Temperature_Sensor,Air_Temperature_Setpoint,Alarm,Angle_Sensor,Average_Zone_Air_Temperature_Sensor,Chilled_Water_Differential_Temperature_Sensor,...,Warmest_Zone_Air_Temperature_Sensor,Water_Flow_Sensor,Water_Temperature_Sensor,Water_Temperature_Setpoint,Wind_Direction_Sensor,Wind_Speed_Sensor,Zone_Air_Dewpoint_Sensor,Zone_Air_Humidity_Sensor,Zone_Air_Humidity_Setpoint,Zone_Air_Temperature_Sensor
0,test_X20367.pkl,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,test_X103084.pkl,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,test_X6910.pkl,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,test_X66332.pkl,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,test_X38528.pkl,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315715,test_X325790.pkl,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
315716,test_X61444.pkl,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
315717,test_X221284.pkl,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
315718,test_X115827.pkl,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
